In [1]:
import pandas
import sqlite3
import requests
import numpy as np
import re
import json
from fake_useragent import UserAgent
from bs4 import BeautifulSoup
from tqdm.auto import tqdm
from nltk.tokenize import wordpunct_tokenize
from pymorphy2 import MorphAnalyzer
from nltk.corpus import stopwords

Данные были собраны с сайта https://spongebob.fandom.com/wiki/SpongeBob_SquarePants_(character) - фандом мультсериала

In [2]:
session = requests.session()
ua = UserAgent(verify_ssl=False)

В списке собраны имена персонажей, которые надо подставить в ссылку для сбора данных

In [33]:
characters = ['SpongeBob_SquarePants_(character)', 
              'Patrick_Star', 
              'Squidward_Tentacles',
              'Eugene_H._Krabs',
              'Sheldon_J._Plankton', 
              'Karen_Plankton', 
              'Sandy_Cheeks', 
              'Gary_the_Snail']

In [5]:
re_del = re.compile(r'\[\d+\]')  # удаляет лишнее
re_bad = re.compile(r'[a-z\]][A-Z]')  # находит сочетания, которые надо разделить

Так как после сбора данных предложения имели не совсем подобающий вид, мне понадобилось сделать функцию, которая делает их немного лучше

In [281]:
def cool_description(sentence):
    new_sent = []
    for word in sentence.split():
        if not word.startswith(('SpongeBob', 'SquarePants')):
            if re_bad.search(word):
                first_word = word[0]
                for i in range(1, len(word)):
                    if word[i].islower():
                        first_word += word[i]
                    else:
                        first_word += '\n'
                        second_word = word[i:]
                        word = first_word + second_word
                        break
            new_sent.append(word)
        else:
            new_sent.append(word)
    return re_del.sub('', ' '.join(new_sent))

Функция, которая собирает данные для каждого персонажа: gen_info - основная информация по разным разделам (образование, цвет глаз и тд), short_info - краткое описание персонажа, chat_name - имя персонажа по статье

In [6]:
def get_information(name):
    link = f'https://spongebob.fandom.com/wiki/{name}'
    req = session.get(link, headers={'User-Agent': ua.random})
    page = req.text
    soup = BeautifulSoup(page, 'html.parser')
    all_info = soup.find_all('p')
    gen_info = {}
    gen_info_all = all_info[0].text.strip().split('\n')
    char_name = gen_info_all[0]
    for i, info in enumerate(gen_info_all):
        if (not info.startswith('\t')) and (info != ''):
            if (':' in info) and (len(info) < 30):
                if info == 'Portrayer:':
                    break
                decription = cool_description(gen_info_all[i+1])
                gen_info[info] = decription
    for i in range(len(all_info)):
        text = all_info[i].text
        if text.startswith(char_name[:3]):
            short_info = re_del.sub('', text)
            break
    return char_name, gen_info, short_info

In [283]:
personal_info = {}
for char in characters:
    char_name, gen_info, short_info = get_information(char)
    personal_info[char_name] = {'gen_info': gen_info, 'short_info': short_info}

In [288]:
with open('personal_info.json', 'w') as f:
    f.write(json.dumps(personal_info))